In [1]:
!pip install loguru


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.6 MB/s eta 0:00:00


In [2]:
import shutil
import os, sys
import json

shutil.rmtree('/content/cen_sierra_pywr_new', ignore_errors=True)
!git clone https://github.com/Maburidi/cen_sierra_pywr_new.git
sys.path.insert(0,'/content/cen_sierra_pywr_new/')

%cd /content/cen_sierra_pywr_new/


Cloning into 'cen_sierra_pywr_new'...
remote: Enumerating objects: 15533, done.
remote: Counting objects: 100% (1233/1233), done.
remote: Compressing objects: 100% (760/760), done.
remote: Total 15533 (delta 832), reused 432 (delta 432), pack-reused 14300 (from 4)
Receiving objects: 100% (15533/15533), 955.81 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (7234/7234), done.
Updating files: 100% (1639/1639), done.
/content/cen_sierra_pywr_new


In [3]:

import os
import multiprocessing as mp
import argparse
import shutil
from pathlib import Path
import pandas as pd
from joblib import Parallel, delayed
import json
from loguru import logger

from sierra.utilities import simplify_network


def create_full_natural_flow(src, dst):
    basin_runoff_dir = src
    subwats = []
    for filename in os.listdir(basin_runoff_dir):
        if '.csv' not in filename:
            continue
        path = os.path.join(basin_runoff_dir, filename)
        df = pd.read_csv(path, parse_dates=True, index_col=0, header=0)
        subwats.append(df)
    df = pd.concat(subwats, axis=1).sum(axis=1).to_frame()
    df.index.name = 'date'
    df.columns = ['flow']

    outdir = dst
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    # daily
    df.to_csv(os.path.join(outdir, 'full_natural_flow_daily_mcm.csv'))
    # (df/0.0864*35.315).to_csv(os.path.join(outdir, 'full_natural_flow_daily_cfs.csv'))

    # monthly
    df2 = df.resample('MS').sum()
    df2.to_csv(os.path.join(outdir, 'full_natural_flow_monthly_mcm.csv'))

    # annual
    df3 = df2.copy()
    df3['WY'] = [d.year if d.month <= 9 else d.year + 1 for d in df2.index]
    df3 = df3.reset_index().set_index('WY').drop('date', axis=1).groupby('WY').sum()
    df3.to_csv(os.path.join(outdir, 'full_natural_flow_annual_mcm.csv'))

    return


def aggregate_subwatersheds(scenario_path, basin):
    if not os.path.exists(scenario_path):
        raise Exception('Scenario path "{}" does not exist'.format(scenario_path))

    basin_runoff_dir = os.path.join(scenario_path, 'runoff')

    if not os.path.exists(basin_runoff_dir):
        raise Exception('Basin path "{}" does not exist'.format(basin_runoff_dir))

    subwat_groups = {}

    # collect subwatersheds
    model_path = '/content/cen_sierra_pywr_new/sierra/models/' + basin + '/pywr_model.json'

    with open(model_path) as f:
        full_model = json.load(f)

    model = simplify_network(full_model, scenario_path=scenario_path, delete_gauges=True, delete_observed=True,
                             delete_scenarios=True,
                             aggregate_runoff=False)

    for n1, n2 in model['edges']:
        if ' Headflow' in n1:
            subwat_groups[n2] = subwat_groups.get(n2, []) + [n1]

    runoff_data = []
    print(subwat_groups.items())
    for subwat_group, subwats in subwat_groups.items():
        subwat_group_name = '{} Inflow'.format(subwat_group)
        subwat_group_runoff = []
        for subwat in subwats:
            filename = 'tot_runoff_sb{}_mcm.csv'.format(subwat.split(' ')[0].split('_')[1])
            path = os.path.join(scenario_path, 'runoff', filename)
            df = pd.read_csv(path, parse_dates=True, index_col=0, header=0)
            subwat_group_runoff.append(df)
        df = pd.concat(subwat_group_runoff, axis=1).sum(axis=1).to_frame()
        df.index.name = 'date'
        df.columns = ["flow"]

        outdir = os.path.join(scenario_path, 'runoff_aggregated')
        if not os.path.exists(outdir):
            os.makedirs(outdir)

        df.to_csv(os.path.join(outdir, '{} mcm.csv'.format(subwat_group_name)))

    return





def create_forecasted_hydrology(scenario_path, dataset=None, default_alpha=0.2, nyears_of_record=25):

    alphas = {}  # Note: default is zero
    for m in range(1, 13):
        alphas[m] = {}
        if 3 >= m <= 9:  # Mar-Sep
            for m2 in range(m, 9 + 1):
                alphas[m][m2] = 0.5 if m == 3 else 0.9

    # Initial pre-processing

    debug = False
    month_columns = ['{:02}'.format(i) for i in range(1, 13)]

    # get source runoff data
    runoff_dir = 'runoff_aggregated'
    scenario_runoff_dir_path = os.path.join(scenario_path, runoff_dir)
    if dataset in ['historical', 'gcms', 'LOCA2_gcms']:
        src_runoff_dir_path = scenario_runoff_dir_path
    else:
        src_runoff_dir_path = os.sep.join(scenario_path.split(os.sep)[:-2] + ['historical', 'Livneh', runoff_dir])
    src_runoff_filenames = os.listdir(src_runoff_dir_path)

    # create output data folder
    # runoff_dir_path = os.path.join(scenario_path, runoff_dir)
    #         print(runoff_dir)
    # runoff_dir_monthly = runoff_dir_path.replace(runoff_dir, 'runoff_monthly')
    runoff_dir_monthly_forecasts = os.path.join(scenario_path, 'runoff_monthly_forecasts')
    if os.path.exists(runoff_dir_monthly_forecasts):
        shutil.rmtree(runoff_dir_monthly_forecasts, ignore_errors=True)
    os.makedirs(runoff_dir_monthly_forecasts)

    months_to_calculate = None

    for runoff_filename in src_runoff_filenames:
        if '.csv' not in runoff_filename:
            continue
        src_path = os.path.join(src_runoff_dir_path, runoff_filename)
        logger.info('Processing ' + src_path)
        df = pd.read_csv(src_path, parse_dates=True, index_col=0)
        col = df.columns[0]

        # Aggregate to monthly
        df2 = df.groupby([lambda x: x.year, lambda x: x.month]).sum()
        df2.index.names = ['year', 'month']

        if months_to_calculate is None:
            if dataset != 'sequences':
                months_to_calculate = list(df2.index)
                earliest_year = months_to_calculate[0][0]
            elif dataset == 'sequences':
                n_years = int(scenario_path.split('_Y')[1][:2])
                print(n_years)
                months_to_calculate = []
                for wy in range(2025, 2025 + n_years):
                    for m in [10,11,12,1,2,3,4,5,6,7,8,9]:
                        y = wy - 1 if m >= 10 else wy
                        months_to_calculate.append((y, m))
                earliest_year = 2013 - nyears_of_record
            else:
                raise('Routine not complete for dataset {}'.format(dataset))

        vals = []
        for i, (year, month) in enumerate(months_to_calculate):

            # Monthly median
            start_year = max(year - nyears_of_record, earliest_year)
            end_year = start_year + nyears_of_record
            years_of_record = list(range(start_year, end_year + 1))
            index_slice = pd.IndexSlice[years_of_record, :]
            df3 = df2.loc[index_slice]
            df_median = df3.groupby('month').median()

            q_next = df2[col].iloc[i:i + 12].values
            if len(q_next) < 12:
                break

            next_months = [i + month for i in range(12)]
            next_months = [m if m < 13 else m - 12 for m in next_months]
            q_next_median = [df_median[col].loc[m] for m in next_months]

            # CORE FORECASTING ROUTINE
            next_months_qfcst = []
            for j, m in enumerate(next_months):
                alpha = alphas[month].get(m, default_alpha)
                fcst = alpha * q_next[j] + (1 - alpha) * q_next_median[j]
                next_months_qfcst.append(fcst)

            vals.append(next_months_qfcst)

        index = pd.to_datetime(['{}-{}-01'.format(ym[0], ym[1]) for ym in months_to_calculate])
        df_final = pd.DataFrame(index=index[:len(vals)], data=vals, columns=month_columns)
        df_final.index.name = 'Date'
        df_final.to_csv(os.path.join(runoff_dir_monthly_forecasts, runoff_filename))

        if debug:
            #             print(df3.head())
            #             fig, ax = plt.subplots(figsize=(12,5))
            #             df3.plot(ax=ax)
            #             plt.show()
            break


def calculate_WYT_P2005_P2130(scenario_path):
    def assign_WYT(row):
        if row['flow'] <= 350000:
            x = 1
        elif row['flow'] <= 676000:
            x = 2
        elif row['flow'] <= 1050000:
            x = 3
        elif row['flow'] <= 1585000:
            x = 4
        else:
            x = 5
        return x

    def assign_WY(row):
        if row['date'].month in [10, 11, 12]:
            x = row['date'].year + 1
        else:
            x = row['date'].year
        return x

    fnf_path = os.path.join(scenario_path, 'preprocessed', 'full_natural_flow_daily_mcm.csv')
    inflow_melones = pd.read_csv(fnf_path, names=['date', 'flow'], header=0, parse_dates=[0])  # mcm
    inflow_melones['WY'] = inflow_melones.apply(lambda row: assign_WY(row), axis=1)
    df = inflow_melones.groupby('WY')['flow'].sum() * 810.71318  # mcm to AF
    df1 = df.to_frame()
    df1['WYT'] = df1.apply(lambda row: assign_WYT(row), axis=1)
    df1.drop('flow', axis=1, inplace=True)
    out_path = os.path.join(scenario_path, 'preprocessed', 'WYT_P2005_P2130.csv')
    df1.to_csv(out_path)


def calculate_WYT_P2019(scenario_path):
    def assign_WYT(row):
        if row['flow'] <= 140000:
            x = 1
        elif row['flow'] <= 320000:
            x = 2
        elif row['flow'] <= 400000:
            x = 3
        elif row['flow'] <= 500000:
            x = 4
        else:
            x = 5
        return x

    fnf_path = os.path.join(scenario_path, 'preprocessed', 'full_natural_flow_daily_mcm.csv')
    df = pd.read_csv(fnf_path, names=['date', 'flow'], index_col=0, header=0, parse_dates=[0])  # mcm
    df['year'] = df.index.year
    df1 = df[df.index.map(lambda x: x.month in [4, 5, 6, 7])]
    df2 = df1.groupby('year').sum() / 1.2335 * 1000
    df2.index.name = 'WY'
    df2['WYT'] = df2.apply(lambda row: assign_WYT(row), axis=1)
    df2.drop('flow', axis=1, inplace=True)
    outpath = os.path.join(scenario_path, 'preprocessed', 'WYT_P2019.csv')
    df2.to_csv(outpath)

def calculate_peak_donnell_lake_inflow(scenario_path):
    dfs = []
    for subwat in [18, 19, 20, 21]:
        path = os.path.join(scenario_path, 'runoff', 'tot_runoff_sb{}_mcm.csv'.format(subwat))
        df = pd.read_csv(path, header=0, index_col=0, parse_dates=True, names=['date', 'flow'])
        dfs.append(df)
    df = pd.concat(dfs)
    df['year'] = df.index.year
    grouped_df = df[df.index.map(lambda x: x.month in [3, 4, 5])].groupby('year')
    peak_flow_df = grouped_df.idxmax()
    peak_flow_df.columns = ['date']
    # peak_flow_df['flow'] = grouped_df.max()

    outpath = os.path.join(scenario_path, 'preprocessed', 'Donnells_Reservoir_Peak_MAM_Runoff_date.csv')
    peak_flow_df.to_csv(outpath)

    return


import os
from datetime import datetime, timedelta
from itertools import product
import pandas as pd


def full_natural_flow_exceedance_forecast(scenario_path):
    """
    This function calculates the full natural flow exceedance forecasts each month.
    Actually, for now it really does no such thing, and simply assumes perfect forecast. However, it can be updated
    with a real forecasting routine as needed. For now, the focus is on the Stanislaus River, Feb-Sep runoff.

    :param basin_preprocessed_path:
    :param scenario:
    :return:
    """


    # Approximate DWR's 50% exceedance flows for the basin.
    if 'sequences' in scenario_path:
        livneh_scenario_path = scenario_path.split('sequences')[0] + 'historical/Livneh'
        fnf_path = os.path.join(livneh_scenario_path, 'preprocessed', 'full_natural_flow_daily_mcm.csv')
    else:
        fnf_path = os.path.join(scenario_path, 'preprocessed', 'full_natural_flow_daily_mcm.csv')
    fnf_df = pd.read_csv(fnf_path, index_col=0, header=0, parse_dates=True)

    months = list(range(3, 9 + 1))
    years = sorted(list(set([dt.year for dt in fnf_df.index])))[1:]
    exceedances = [50]
    months_exceedances = list(product(months, exceedances))
    years_months = list(product(years, months))
    columns = pd.MultiIndex.from_tuples(months_exceedances)
    index = pd.MultiIndex.from_tuples(years_months)
    fnf_all = pd.DataFrame(index=index, columns=columns)
    fnf_all.index.names = ['year', 'month']
    fnf_all.columns.names = ['month', 'exceedance']
    for exceedance in exceedances:
        for year, month in years_months:

            ytd_monthly = []

            # forecast year-to-go
            ytg_start = '{:04}-{:02}-01'.format(year, month)
            ytg_end = '{:04}-09-30'.format(year)

            if month <= 6:
                # insert forecast routine here
                ytg_monthly = fnf_df[ytg_start:ytg_end].resample('MS').sum()['flow'].values

            else:
                # assume perfect year-to-go forecast after June
                ytg_monthly = fnf_df[ytg_start:ytg_end].resample('MS').sum()['flow'].values

            # actual to-date
            if month > 3:
                ytd_start = '{:04}-03-01'.format(year)
                ytd_end = (datetime(year, month, 1) - timedelta(days=1)).strftime('%Y-%m-%d')
                ytd_monthly = fnf_df[ytd_start:ytd_end].resample('MS').sum()['flow'].values

            # aggregated
            ytd_ytg = list(ytd_monthly) + list(ytg_monthly)
            fnf_all.loc[(year, month), (months, exceedance)] = ytd_ytg

        # sum across exceedance
        fnf_all[('sum', exceedance)] = fnf_all.loc[(years, months), (months, exceedance)].sum(axis=1)

    outpath = os.path.join(scenario_path, 'preprocessed', 'exceedance_forecast_mcm.csv')
    fnf_all.to_csv(outpath)




FRACTIONS = dict(a=0.42, b=0.03, c=0.47, d=0.08)


def disaggregate_SJN_09_subwatershed(scenario_path):
    """
    This scripts disaggregates the original SJN_09 subwatersheds into four smaller subcatchments.
    :param scenarios_path:
    :param scenarios:
    :return:
    """

    hydrology_path = os.path.join(scenario_path, 'runoff')

    sjn09_path = os.path.join(hydrology_path, 'tot_runoff_sb09_mcm.csv')
    df = pd.read_csv(sjn09_path, index_col=0, header=0, parse_dates=True)

    for subsubwat in ['a', 'b', 'c', 'd']:
        df2 = df * FRACTIONS[subsubwat]
        df2.to_csv(sjn09_path.replace('sb09', 'sb09{}'.format(subsubwat)))






def sjrrp_below_friant(src, dst):
    thresholds_taf = [0, 400, 670, 930, 1450, 2500]
    allocations_taf = [116.866, 187.785, 272.280, 330.300, 400.300, 547.400, 673.488]

    inpath = os.path.join(src, 'full_natural_flow_annual_mcm.csv')

    df = pd.read_csv(inpath, index_col=0, header=0)

    wyts = []
    allocations_mcm = []
    allocation_fractions = []
    for i, runoff_mcm in enumerate(df['flow']):
        runoff_taf = runoff_mcm / 1.2335
        wyt = sum([1 for t in thresholds_taf if runoff_taf > t])
        wyts.append(wyt)

        idx = wyt-1
        if wyt in [1, 2, 6]:
            if wyt == 1:
                allocation_taf = allocations_taf[0]
            elif wyt == 2:
                allocation_taf = allocations_taf[1]
            else:
                allocation_taf = allocations_taf[-1]
            allocation_fraction = 1
        else:
            tlow = thresholds_taf[idx]
            thigh = thresholds_taf[idx+1]
            rlow = allocations_taf[idx]
            rhigh = allocations_taf[idx+1]
            slope = (rhigh-rlow)/(thigh-tlow)
            allocation_taf = rlow + slope * (runoff_taf - tlow)
            allocation_taf_middle = (rhigh + rlow) / 2
            allocation_fraction = allocation_taf / allocation_taf_middle

        allocation_mcm = allocation_taf * 1.2335
        allocations_mcm.append(allocation_mcm)
        allocation_fractions.append(allocation_fraction)

    df2 = pd.DataFrame(index=df.index)
    df2['WYT'] = wyts
    df2['Annual allocation mcm'] = allocations_mcm
    df2['Allocation adjustment'] = allocation_fractions

    outpath = os.path.join(dst, 'SJ restoration flows.csv')
    df2.to_csv(outpath)

def calculate_millerton_snowmelt_inflow(source, destination):

    # Read in the data
    inpath = os.path.join(source, 'full_natural_flow_daily_mcm.csv')
    df = pd.read_csv(inpath, index_col=0, header=0, parse_dates=True) / 1.2335 * 1000  # mcm to acre-feet

    # Filter by month (Apr-Jul) and resample by year
    df2 = df[df.index.map(lambda x: x.month in [4, 5, 6, 7])].resample('Y').sum()
    df2.index = df2.index.map(lambda x: x.year)
    df2.index.name = 'year'

    outpath = os.path.join(destination, 'inflow_MillertonLake_AprToJul_af.csv')
    df2.to_csv(outpath)

    return

def calc_WY(flow):
    if flow >= 450:
        return 1
    else:
        return 2

def calculate_Exchequer_WYT(scenario_path):
    fnf_path = os.path.join(scenario_path, 'preprocessed', 'full_natural_flow_daily_mcm.csv')
    df = pd.read_csv(fnf_path, header=0, index_col=0, parse_dates=True, names=['flow'])
    df2 = df[(df.index.month >= 4) & (df.index.month <= 7)]
    df3 = df2.resample('Y').sum()
    df3 /= 1.2335  # mcm to TAF

    # compute the water year type
    df3['WYT'] = df3['flow'].apply(lambda x: 1 if x >= 450 else 2)
    df3.index = df3.index.year
    df3.index.name = 'WY'

    # create the final WYT dataframe, adding an initial WYT to the beginning
    wyt_df = pd.Series(index=[df3.index[0] - 1] + list(df3.index))
    wyt_df[wyt_df.index[0]] = 3  # assume the previous water year's type is 3
    wyt_df.values[1:] = df3['WYT']
    wyt_df.index.name = 'WY'
    wyt_df.name = 'WYT'

    outpath = os.path.join(scenario_path, 'preprocessed', 'Exchequer_WYT.csv')
    wyt_df.to_csv(outpath)




basins = ['Stanislaus', 'Merced', 'Upper_San_Joaquin', 'Tuolumne']

def calculate_sjvi(dataset, climate):
    root_dir = '/content/cen_sierra_pywr_new/data'
    dfs = []
    for basin in basins:
        full_basin_name = '{}_River'.format(basin.title())

        path = os.path.join(root_dir, full_basin_name, 'hydrology',dataset, climate, 'preprocessed',
                            'full_natural_flow_daily_mcm.csv')
        df = pd.read_csv(path, index_col=0, header=0, parse_dates=True)
        dfs.append(df)

    df_maf = pd.concat(dfs, axis=1).sum(axis=1) / 1.2335 / 1e3

    sjvi = 3.5
    water_years = df_maf.resample('Y').sum().index.year
    sjvi_values = pd.Series(index=water_years, name='SJVI (maf)', dtype=np.float64)
    sjvi_values.index.name = 'Water Year'
    sjvi_values[water_years[0]] = sjvi
    for wy in water_years[1:]:
        apr_1 = '{}-04-01'.format(wy)
        jul_31 = '{}-07-31'.format(wy)
        oct_1 = '{}-10-01'.format(wy - 1)
        mar_31 = '{}-03-31'.format(wy)
        apr_jul_maf = df_maf[apr_1:jul_31].sum()
        oct_mar_maf = df_maf[oct_1:mar_31].sum()
        sjvi = 0.6 * apr_jul_maf + 0.2 * oct_mar_maf + 0.2 * sjvi
        sjvi_values[wy] = sjvi

    outpath = os.path.join(root_dir, 'common', 'hydrology', dataset, climate)
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    sjvi_values.to_csv(os.path.join(outpath, 'SJVI.csv'))
    return


In [ ]:
########################## Pre-Preocessing for Tuolumne River #################################
###############################################################################################

# [1] ============= Aggregating subwatersheds ---------> inputted folder: runoff / Generated folder: runoff_aggregated

gcm_model = "ACCESS_CM2"
basin = 'tuolumne'                # Upper_san_joaquin, tuolumne, stanislaus, merced
basin0 = 'Tuolumne_River'         # Upper_San_Joaquin_River, Stanislaus_River, Merced_River, Tuolumne_River


scenario_path = '/content/cen_sierra_pywr_new/data/' + basin0 + '/hydrology/LOCA2_gcms/' + gcm_model

aggregate_subwatersheds(scenario_path, basin)


# [2] ============= Creating forecasted hydrology ---------> inputted folder: runoff_aggregated  /  generates folder: runoff_monthly_forecasts


scenario_path = '/content/cen_sierra_pywr_new/data/'+ basin0 + '/hydrology/LOCA2_gcms/' +  gcm_model
create_forecasted_hydrology(scenario_path, dataset='LOCA2_gcms', default_alpha=0.2, nyears_of_record=25)



# [3] ============ Creating full natural flow ======> inputted folder: runoff / generated folder: preprocessed

src = os.path.join(scenario_path, 'runoff_aggregated')
preprocessed_path = os.path.join(scenario_path, 'preprocessed')
dst = preprocessed_path

# run the function
create_full_natural_flow(src, dst)



In [10]:
########################## Pre-Preocessing for MERCED River ###################################
###############################################################################################

gcm_model = "ACCESS_CM2"
basin = 'merced'                # Upper_san_joaquin, tuolumne, stanislaus, merced
basin0 = 'Merced_River'         # Upper_San_Joaquin_River, Stanislaus_River, Merced_River, Tuolumne_River
scenario_path = '/content/cen_sierra_pywr_new/data/' + basin0 + '/hydrology/LOCA2_gcms/' + gcm_model



# [1] ============= Aggregating subwatersheds ---------> inputted folder: runoff / Generated folder: runoff_aggregated

aggregate_subwatersheds(scenario_path, basin)

# [2] ============= Creating forecasted hydrology ---------> inputted folder: runoff_aggregated  /  generates folder: runoff_monthly_forecasts

create_forecasted_hydrology(scenario_path, dataset='LOCA2_gcms', default_alpha=0.2, nyears_of_record=25)

# [3] ============ Creating full natural flow ======> inputted folder: runoff / generated folder: preprocessed

src = os.path.join(scenario_path, 'runoff_aggregated')
preprocessed_path = os.path.join(scenario_path, 'preprocessed')
dst = preprocessed_path

# run the function
create_full_natural_flow(src, dst)

# [4] ============ calculate_Exchequer_WYT =====> inputted folder: preprocessed / generated file: preprocessed/Exchequer_WYT.csv

calculate_Exchequer_WYT(scenario_path)



dict_items([])


In [ ]:
########################## Pre-Preocessing for Stainslus River ###################################
###############################################################################################

# [1] ============= Aggregating subwatersheds ---------> inputted folder: runoff / Generated folder: runoff_aggregated

gcm_model = "ACCESS_CM2"
basin = 'stanislaus'                # Upper_san_joaquin, tuolumne, stanislaus, merced
basin0 = 'Stanislaus_River'         # Upper_San_Joaquin_River, Stanislaus_River, Merced_River, Tuolumne_River


scenario_path = '/content/cen_sierra_pywr_new/data/' + basin0 + '/hydrology/LOCA2_gcms/' + gcm_model

aggregate_subwatersheds(scenario_path, basin)

# [2] ============= Creating forecasted hydrology ---------> inputted folder: runoff_aggregated  /  generates folder: runoff_monthly_forecasts

create_forecasted_hydrology(scenario_path, dataset='LOCA2_gcms', default_alpha=0.2, nyears_of_record=25)

# [3] ============ Creating full natural flow ======> inputted folder: runoff / generated folder: preprocessed

src = os.path.join(scenario_path, 'runoff_aggregated')
preprocessed_path = os.path.join(scenario_path, 'preprocessed')
dst = preprocessed_path
create_full_natural_flow(src, dst)

# [4] ============ calculate other files =====> inputted folder: preprocessed / generated file are in: preprocessed

calculate_WYT_P2005_P2130(scenario_path)
calculate_WYT_P2019(scenario_path)
calculate_peak_donnell_lake_inflow(scenario_path)
full_natural_flow_exceedance_forecast(scenario_path)



In [19]:
########################## Pre-Processing for Upper San Joaquin River #########################
###############################################################################################

gcm_model = "ACCESS_CM2"
basin = 'upper_san_joaquin'                # Upper_san_joaquin, tuolumne, stanislaus, merced
basin0 = 'Upper_San_Joaquin_River'         # Upper_San_Joaquin_River, Stanislaus_River, Merced_River, Tuolumne_River


scenario_path = '/content/cen_sierra_pywr_new/data/' + basin0 + '/hydrology/LOCA2_gcms/' + gcm_model

# [0] ============= disaggregate SJN_09 subwatershed:   Inputted folder: runoff / Generated files: four extra files saved in runoff folder

disaggregate_SJN_09_subwatershed(scenario_path)

# [1] ============= Aggregating subwatersheds ---------> inputted folder: runoff / Generated folder: runoff_aggregated

aggregate_subwatersheds(scenario_path, basin)

# [2] ============= Creating forecasted hydrology ---------> inputted folder: runoff_aggregated  /  generates folder: runoff_monthly_forecasts

create_forecasted_hydrology(scenario_path, dataset='LOCA2_gcms', default_alpha=0.2, nyears_of_record=25)

# [3] ============ Creating full natural flow ======> inputted folder: runoff / generated folder: preprocessed

src = os.path.join(scenario_path, 'runoff_aggregated')
preprocessed_path = os.path.join(scenario_path, 'preprocessed')
dst = preprocessed_path
create_full_natural_flow(src, dst)

# [4] ============ calculate other files =====> inputted folder: preprocessed / generated file are in: preprocessed
src = dst = preprocessed_path
sjrrp_below_friant(src, dst)
calculate_millerton_snowmelt_inflow(src, dst)


In [31]:
# rename files in folders, like - by _ and . ny nothing and " " by "_"

directory = "/content/cen_sierra_pywr_new/data/Upper_San_Joaquin_River/hydrology/LOCA2_gcms/ACCESS_CM2/runoff_aggregated"

for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
        name, ext = os.path.splitext(filename)
        new_name = name.replace(" ", "_").replace(".", "").replace("-", "_") + ext
        os.rename(
            os.path.join(directory, filename),
            os.path.join(directory, new_name)
        )

print("Renaming completed!")


Renaming completed!


In [34]:
# Save the contents

from google.colab import files

shutil.make_archive('preprocessed', 'zip', '/content/cen_sierra_pywr_new/data/Upper_San_Joaquin_River/hydrology/LOCA2_gcms/ACCESS_CM2/preprocessed')
files.download('preprocessed.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
########################### Calculate the SJVI INDEX ################################

dataset = "LOCA2_gcms"
climate = "ACCESS_CM2"
calculate_sjvi(dataset, climate)


<ipython-input-42-acf206a28fc6>:21: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  water_years = df_maf.resample('Y').sum().index.year
